# Step1: Prepare DWI Files
* record the moving volumes
* and arrange best b0 at the beginning

## Functions Definition

In [ ]:
def mydcm2niix(Study_folder,replace_empty,dcm,nifti):
    '''
    Function 1: replace the ' ' in the subfolder name with '_';
    Function 2: make the '*.dcm' files to '*.nii'
    '''
    import os
    import numpy as np
    Study_folder = Study_folder
    os.chdir(Study_folder)
    original_names = os.listdir(Study_folder)
    # final names of each folder
    final_names = os.listdir(Study_folder)
    for i in np.arange(len(final_names)):
        for j in np.arange(len(final_names[i])):
            if final_names[i][j] == ' ':
                temp_name = list(final_names[i])
                temp_name[j] = '_'
                final_names[i] = "".join(temp_name)
    # replace empty space with '_'
    if replace_empty == True:
        for i in range(len(original_names)):
            os.rename(original_names[i],final_names[i])
        # create nifti or dicom
        if nifti == True:
            for i in range(len(final_names)):
                os.chdir(os.path.join(Study_folder,final_names[i]))
                # !rm *.json *.nii *.bval *.bvec
                !dcm2niix ./
                if dcm == False:
                    !rm *.dcm
                elif dcm == True:
                    print('dicoms kept!')
        if nifti == False:
            print('no files have been converted to nifti' )
    else:
        print('Error! No operation has been done!')
    

In [ ]:
def delete_vol(brain_path,input_name,output_name,remove_vols,b_name_in,b_name_out,b_remove,type_output):
    '''
    delete_vol(brain_path,input_name='dwi_0.nii',output_name='dwi.nii',remove_vols=[9,16],
                b_name_in='dwi_0',b_name_out='dwi',b_remove=True,type_output=16)
    '''
    import nibabel as nib
    import numpy as np
    from dipy.io.image import load_nifti,save_nifti
    import os
    from tqdm import tqdm
    if type_output == 16:
        type_output = np.int16
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        # load data
        bvals = np.loadtxt(b_name_in+'.bval')
        bvecs = np.loadtxt(b_name_in+'.bvec')
        data,aff,img = load_nifti(input_name,return_img=True)
        zooms_old = img.header.get_zooms()[0:4]
        data_save = np.zeros(data.shape[0:3]+(data.shape[3]-len(remove_vols),))
        bval_new = np.zeros((1,data.shape[3]-len(remove_vols)))
        bvec_new = np.zeros((3,data.shape[3]-len(remove_vols)))
        if b_remove == True:
            i_vol_new = -1
            for i_vol in np.arange(data.shape[3]):
                if i_vol not in np.array(remove_vols)-1:
                    i_vol_new = i_vol_new + 1
                    data_save[:,:,:,i_vol_new] = data[:,:,:,i_vol]
                    if b_remove == True:
                        bval_new[0,i_vol_new] = bvals[i_vol]
                        bvec_new[:,i_vol_new] = bvecs[:,i_vol]
            img_new = nib.Nifti1Image(data_save.astype(type_output), np.eye(4))
            img_new.header.set_zooms(zooms_old)
            img_new.header['intent_code'] = 1005
            img_new.header['sform_code'] = 1
            img_new.header['qform_code'] = 1
            img_new.header['qoffset_x'] = 0
            img_new.header['qoffset_y'] = 0
            img_new.header['qoffset_z'] = 0
            img_new.header['dim'][4] = data_save.shape[3]#[4,128,128,14,6,1,0,0]
        else:
            bval_new = bvals
            bvec_new = bvecs
            data_save = data
        #save new nifti file
        img.header['intent_code'] = 1005
        save_nifti(output_name, data_save.astype(type_output), aff,hdr=img.header)
        np.savetxt(b_name_out+'.bval', bval_new, fmt='%1.5f')
        np.savetxt(b_name_out+'.bvec', bvec_new, fmt='%1.5f')

In [ ]:
def best_b0_first(brain_path,input_name,output_name,best_b0_vols,b0_threshold,b_name_in,b_name_out,type_output):
    '''
    best_b0_vols: should be one number showing which volume is the best b0 volume; the best b0 index equals to best_b0_vols-1;
    b0_threshold: one number value; b0 values are [0,b0_threshold);
    usage: best_b0_first(brain_path,input_name='dwi_0.nii',output_name='dwi.nii',best_b0_vols=[9],b0_threshold=10,
                  b_name_in='dwi_0',b_name_out='dwi',type_output=16)
    '''
    import nibabel as nib
    import numpy as np
    from dipy.io.image import load_nifti,save_nifti
    import os
    from tqdm import tqdm
    if type_output == 16:
        type_output = np.int16
    for Study_folder in tqdm(brain_path):
        os.chdir(Study_folder)
        # load data
        bvals = np.loadtxt(b_name_in+'.bval')
        bvecs = np.loadtxt(b_name_in+'.bvec')
        data,aff,img = load_nifti(input_name,return_img=True)
        if len(data.shape)!=4:
            print('Error!!! Data dimention must be 4D, plese check your data dimention!')
        else:
            zooms_old = img.header.get_zooms()[0:4]
            data_new = np.zeros(data.shape)
            bval_new = np.zeros((1,data.shape[3]))
            bvec_new = np.zeros((3,data.shape[3]))
            # best b0 vol
            index_best_b0 = best_b0_vols-1
            index_b0 = np.where(bvals<b0_threshold)[0]
            index_b0_notbest = index_b0[np.where(index_b0!=index_best_b0)[0]]
            index_b0_final = np.concatenate(([index_best_b0],index_b0_notbest),axis =0)
            index_nonb0_final = np.where(bvals>=b0_threshold)[0]
            index_final = np.concatenate((index_b0_final,index_nonb0_final),axis =0)
            # arange the volumes
            num_b0 = len(index_b0_final)
            data_new = data[:,:,:,index_final]
            # arange b values and b vectors
            bval_new = bvals[index_final][np.newaxis,:]
            bvec_new = bvecs[:,index_final]
            # save new nifti file
            img.header['intent_code'] = 1005
            save_nifti(output_name, data_new.astype(type_output), aff,hdr=img.header)
            np.savetxt(b_name_out+'.bval', bval_new, fmt='%1.5f')
            np.savetxt(b_name_out+'.bvec', bvec_new, fmt='%1.5f')

## Processing

### Change folder name and generate nifti files

In [ ]:
#-----------------------------------------------------------------
# Change folder names
Study_folder = '/media/erjun/One_Touch/Newborns/sub-B029S3/dicom/'
mydcm2niix(Study_folder=Study_folder,replace_empty=True,dcm=True,nifti=False)
# Change dicom files to nifti
Study_folder = '/media/erjun/One_Touch/Newborns/sub-B029S3/nifti/'
mydcm2niix(Study_folder=Study_folder,replace_empty=True,dcm=False,nifti=True)

### Removing Volumes If Needed

In [ ]:
# Delete moving volumes
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-02/Diffusion1/']
delete_vol(brain_path,input_name='dwi_original.nii',output_name='dwi_rm.nii',remove_vols=[],
           b_name_in='dwi_original',b_name_out='dwi_rm',b_remove=True,type_output=16)

### Moving Best Quality b0 To the Beginning

In [ ]:
brain_path = ['/media/erjun/One_Touch/Newborns/sub-B027S1/ses-02/Diffusion1/']
best_b0_first(brain_path,input_name='dwi_rm.nii',output_name='dwi_mb0.nii',best_b0_vols=3,b0_threshold=11,
                  b_name_in='dwi_rm',b_name_out='dwi_mb0',type_output=16)